## Final Project

### Introduction
In this final project, available to Verified learners only, we'll attempt to predict the type of physical activity (e.g., walking, climbing stairs) from tri-axial smartphone accelerometer data. Smartphone accelerometers are very precise, and different physical activities give rise to different patterns of acceleration.

(Note on project availability: while project submission is only available to Verified learners, all learners are welcome to work on the project on their own and have access to the instructions describing the project.)

### Input Data
The input data used for training in this project consists of two files. The first file, train_time_series.csv, contains the raw accelerometer data, which has been collected using the Beiwe research platform, and it has the following format:
```
timestamp, UTC time, accuracy, x, y, z
```
You can use the timestamp column as your time variable; you'll also need the last three columns, here labeled x, y, and z, which correspond to measurements of linear acceleration along each of the three orthogonal axes.

The second file, train_labels.csv, contains the activity labels, and you'll be using these labels to train your model. Different activities have been numbered with integers. We use the following encoding: 1 = standing, 2 = walking, 3 = stairs down, 4 = stairs up. Because the accelerometers are sampled at high frequency, the labels in train_labels.csv are only provided for every 10th observation in train_time_series.csv.

### Activity Classification
Your goal is to classify different physical activities as accurately as possible. To test your code, you're also provided a file called test_time_series.csv, and at the end of the project you're asked to provide the activity labels predicted by your code for this test data set. Only the course staff have the corresponding true labels for the test data, and the accuracy of your code will be determined as the percentage of correct classifications. Note that in both cases, for training and testing, the input file consists of a single (3-dimensional) time series. To test the accuracy of your code, you'll be asked to upload your predictions as a CSV file. This file called test_labels.csv is provided to you, but it only contains the time stamps needed for prediction; you'll need to augment this file by adding the corresponding class predictions (1,2,3,4).

### Code Run Time
In addition to providing the predictions, you're also asked to time the running time of your code, starting at the moment when you load in the test data set and ending at the moment you're done computing your predictions. You'll be asked to enter this running time, and the goal is to see how fast your code runs compared to the code of others. Because computing speeds vary for several reasons, including hardware and implementation of the code, these numbers aren't directly comparable, and for this reason your grading will not be affected by them. However, it may still be interesting to you to see how long the code of other participants takes to solve the problem.

### Project Submission
You're expected to implement your solution using a Jupyter notebook. Once you're done, you're asked to upload the notebook, which will be peer reviewed by other course participants. This review will impact your final course grade, so you should write your code as clearly code as possible, include comments, and use meaningful variable names.

You can approach this problem any way you'd like. It may be beneficial to search the web for possible solutions, or you may try to solve this problem from scratch. We recommend that you build your code in stages, encapsulating different parts (tasks) of the problem into functions. There are many ways to solve this problem. Good luck!

#### Reading the Data

In [72]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [73]:
train_time = pd.read_csv("train_time_series.csv", index_col=0)
train_time

,timestamp,UTC time,accuracy,x,y,z
20586,1565109930787,2019-08-06T16:45:30.787,unknown,-0.006485,-0.934860,-0.069046
20587,1565109930887,2019-08-06T16:45:30.887,unknown,-0.066467,-1.015442,0.089554
20588,1565109930987,2019-08-06T16:45:30.987,unknown,-0.043488,-1.021255,0.178467
20589,1565109931087,2019-08-06T16:45:31.087,unknown,-0.053802,-0.987701,0.068985
20590,1565109931188,2019-08-06T16:45:31.188,unknown,-0.054031,-1.003616,0.126450
...,...,...,...,...,...,...
24325,1565110305638,2019-08-06T16:51:45.638,unknown,0.024384,-0.710709,0.030304
24326,1565110305738,2019-08-06T16:51:45.738,unknown,0.487228,-1.099136,-0.015213
24327,1565110305838,2019-08-06T16:51:45.838,unknown,0.369446,-0.968506,0.036713
24328,1565110305939,2019-08-06T16:51:45.939,unknown,0.167877,-0.802826,0.049805


In [74]:
train_labels = pd.read_csv("train_labels.csv", index_col=0)
train_labels

,timestamp,UTC time,label
20589,1565109931087,2019-08-06T16:45:31.087,1
20599,1565109932090,2019-08-06T16:45:32.090,1
20609,1565109933092,2019-08-06T16:45:33.092,1
20619,1565109934094,2019-08-06T16:45:34.094,1
20629,1565109935097,2019-08-06T16:45:35.097,1
...,...,...,...
24289,1565110302030,2019-08-06T16:51:42.030,4
24299,1565110303032,2019-08-06T16:51:43.032,4
24309,1565110304034,2019-08-06T16:51:44.034,4
24319,1565110305037,2019-08-06T16:51:45.037,4


## Manipulating The Data

In [75]:
time_labels=train_time.loc[train_labels.index]
time_labels

,timestamp,UTC time,accuracy,x,y,z
20589,1565109931087,2019-08-06T16:45:31.087,unknown,-0.053802,-0.987701,0.068985
20599,1565109932090,2019-08-06T16:45:32.090,unknown,0.013718,-0.852371,-0.000870
20609,1565109933092,2019-08-06T16:45:33.092,unknown,0.145584,-1.007843,-0.036819
20619,1565109934094,2019-08-06T16:45:34.094,unknown,-0.099380,-1.209686,0.304489
20629,1565109935097,2019-08-06T16:45:35.097,unknown,0.082794,-1.001434,-0.025375
...,...,...,...,...,...,...
24289,1565110302030,2019-08-06T16:51:42.030,unknown,-0.641953,-1.469177,0.301041
24299,1565110303032,2019-08-06T16:51:43.032,unknown,-0.171616,-0.366074,-0.059082
24309,1565110304034,2019-08-06T16:51:44.034,unknown,0.401810,-1.077698,0.258911
24319,1565110305037,2019-08-06T16:51:45.037,unknown,0.330338,-1.470062,0.303894


In [76]:
# for i in time_labels.index:
# 	time_labels["x"][i]=sum(train_time.loc[i-9:i].x)
# 	time_labels["y"][i]=sum(train_time.loc[i-9:i].y)
# 	time_labels["z"][i]=sum(train_time.loc[i-9:i].z)

# time_labels

In [77]:
time_labels["label"] = train_labels["label"]
time_labels

,timestamp,UTC time,accuracy,x,y,z,label
20589,1565109931087,2019-08-06T16:45:31.087,unknown,-0.053802,-0.987701,0.068985,1
20599,1565109932090,2019-08-06T16:45:32.090,unknown,0.013718,-0.852371,-0.000870,1
20609,1565109933092,2019-08-06T16:45:33.092,unknown,0.145584,-1.007843,-0.036819,1
20619,1565109934094,2019-08-06T16:45:34.094,unknown,-0.099380,-1.209686,0.304489,1
20629,1565109935097,2019-08-06T16:45:35.097,unknown,0.082794,-1.001434,-0.025375,1
...,...,...,...,...,...,...,...
24289,1565110302030,2019-08-06T16:51:42.030,unknown,-0.641953,-1.469177,0.301041,4
24299,1565110303032,2019-08-06T16:51:43.032,unknown,-0.171616,-0.366074,-0.059082,4
24309,1565110304034,2019-08-06T16:51:44.034,unknown,0.401810,-1.077698,0.258911,4
24319,1565110305037,2019-08-06T16:51:45.037,unknown,0.330338,-1.470062,0.303894,4


## Model

In [78]:
X_train=time_labels[["x","y","z"]]
X_train

,x,y,z
20589,-0.053802,-0.987701,0.068985
20599,0.013718,-0.852371,-0.000870
20609,0.145584,-1.007843,-0.036819
20619,-0.099380,-1.209686,0.304489
20629,0.082794,-1.001434,-0.025375
...,...,...,...
24289,-0.641953,-1.469177,0.301041
24299,-0.171616,-0.366074,-0.059082
24309,0.401810,-1.077698,0.258911
24319,0.330338,-1.470062,0.303894


In [79]:
Y_train=time_labels["label"]
Y_train

20589    1
20599    1
20609    1
20619    1
20629    1
        ..
24289    4
24299    4
24309    4
24319    4
24329    4
Name: label, Length: 375, dtype: int64

In [80]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(time_labels, test_size=0.3, random_state=1)


In [81]:
X_train=df_train[["x","y","z"]]
X_train

Y_train=df_train["label"]
Y_train

22809    2
21779    4
21389    2
21469    2
22949    3
        ..
22619    2
23139    3
21309    2
22939    3
20959    2
Name: label, Length: 262, dtype: int64

In [82]:
len(X_train)

262

In [83]:
forest_classifier = RandomForestClassifier(random_state=1)
forest_classifier.fit(X_train,Y_train)

RandomForestClassifier(random_state=1)

In [84]:
X_test=df_test[["x","y","z"]]
X_test

Y_test=df_test["label"]
Y_test

21259    2
23769    2
23339    3
23189    3
21489    2
        ..
22209    3
24009    2
21809    4
24269    4
22049    1
Name: label, Length: 113, dtype: int64

In [85]:
y_eval=forest_classifier.predict(X_test)

In [93]:
y_eval

array([2, 2, 2, 2, 2, 4, 2, 2, 2, 1, 3, 2, 2, 2, 1, 2, 2, 3, 2, 2, 1, 1,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 3, 2, 2, 2, 2, 4, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 3, 2, 1, 2,
       3, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2])

In [88]:
Y_test

21259    2
23769    2
23339    3
23189    3
21489    2
        ..
22209    3
24009    2
21809    4
24269    4
22049    1
Name: label, Length: 113, dtype: int64

In [104]:
y_dummy = np.full(113,2)


In [105]:
accuracy_score(y_dummy, Y_test)


0.6017699115044248

In [94]:
accuracy_score(y_eval, Y_test)

0.5929203539823009

In [90]:
from sklearn.metrics import accuracy_score

def accuracy(estimator, X, y):
	predictions = estimator.fit(X, y).predict(X)
	return accuracy_score(y, predictions)

In [91]:
accuracy(forest_classifier,X_train,Y_train)

1.0